In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"adityaekapurwanto","key":"7d3185db0fc4dcc1b1d7b64e9d6ea6f5"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [3]:
!kaggle datasets download -d kukuroo3/body-signal-of-smoking

Dataset URL: https://www.kaggle.com/datasets/kukuroo3/body-signal-of-smoking
License(s): CC0-1.0
  0% 0.00/3.40M [00:00<?, ?B/s]
100% 3.40M/3.40M [00:00<00:00, 84.8MB/s]


In [4]:
!unzip body-signal-of-smoking.zip -d body-signal-of-smoking
!ls body-signal-of-smoking

Archive:  body-signal-of-smoking.zip
  inflating: body-signal-of-smoking/competition_format/x_test.csv  
  inflating: body-signal-of-smoking/competition_format/x_train.csv  
  inflating: body-signal-of-smoking/competition_format/y_test.csv  
  inflating: body-signal-of-smoking/competition_format/y_train.csv  
  inflating: body-signal-of-smoking/smoking.csv  
competition_format  smoking.csv


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder

import seaborn as sns
from keras.layers import Dense, BatchNormalization, Dropout, LSTM
from keras.models import Sequential
from keras.utils import to_categorical
from keras import callbacks

import tensorflow as tf
from tensorflow.keras.optimizers import Adam

In [6]:
df = pd.read_csv("/content/body-signal-of-smoking/smoking.csv")

In [7]:
df.head()

,ID,gender,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),...,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,oral,dental caries,tartar,smoking
0,0,F,40,155,60,81.3,1.2,1.0,1.0,1.0,...,12.9,1.0,0.7,18.0,19.0,27.0,Y,0,Y,0
1,1,F,40,160,60,81.0,0.8,0.6,1.0,1.0,...,12.7,1.0,0.6,22.0,19.0,18.0,Y,0,Y,0
2,2,M,55,170,60,80.0,0.8,0.8,1.0,1.0,...,15.8,1.0,1.0,21.0,16.0,22.0,Y,0,N,1
3,3,M,40,165,70,88.0,1.5,1.5,1.0,1.0,...,14.7,1.0,1.0,19.0,26.0,18.0,Y,0,Y,0
4,4,F,40,155,60,86.0,1.0,1.0,1.0,1.0,...,12.5,1.0,0.6,16.0,14.0,22.0,Y,0,N,0


In [8]:
col_names = ['gender']

for col in col_names:
    print (df[col].value_counts())

gender
M    35401
F    20291
Name: count, dtype: int64


In [9]:
numerical = []
catgcols =[]

for col in df.columns:
  if df[col].dtype=="float64":
    numerical.append(col)
  else:
    catgcols.append(col)

for col in df.columns:
  if col in numerical:
    df[col].fillna(df[col].median(), inplace=True)
  else:
    df[col].fillna(df[col].mode()[0], inplace=True)

In [10]:
le = LabelEncoder()

for col in catgcols:
    df[col] = le.fit_transform(df[col])

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55692 entries, 0 to 55691
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   55692 non-null  int64  
 1   gender               55692 non-null  int64  
 2   age                  55692 non-null  int64  
 3   height(cm)           55692 non-null  int64  
 4   weight(kg)           55692 non-null  int64  
 5   waist(cm)            55692 non-null  float64
 6   eyesight(left)       55692 non-null  float64
 7   eyesight(right)      55692 non-null  float64
 8   hearing(left)        55692 non-null  float64
 9   hearing(right)       55692 non-null  float64
 10  systolic             55692 non-null  float64
 11  relaxation           55692 non-null  float64
 12  fasting blood sugar  55692 non-null  float64
 13  Cholesterol          55692 non-null  float64
 14  triglyceride         55692 non-null  float64
 15  HDL                  55692 non-null 

In [12]:
df.drop(columns=['ID','height(cm)','weight(kg)','waist(cm)','eyesight(left)','eyesight(right)','hearing(left)','hearing(right)','fasting blood sugar','HDL','LDL','Urine protein','serum creatinine','AST','ALT','Gtp','dental caries','tartar'], inplace=True)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55692 entries, 0 to 55691
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   gender        55692 non-null  int64  
 1   age           55692 non-null  int64  
 2   systolic      55692 non-null  float64
 3   relaxation    55692 non-null  float64
 4   Cholesterol   55692 non-null  float64
 5   triglyceride  55692 non-null  float64
 6   hemoglobin    55692 non-null  float64
 7   oral          55692 non-null  int64  
 8   smoking       55692 non-null  int64  
dtypes: float64(5), int64(4)
memory usage: 3.8 MB


In [14]:
X = df.drop("smoking",axis=1)
Y = pd.get_dummies(df['smoking'])

In [15]:
fig = px.histogram(df, x='smoking')

fig.update_layout(title_text='<b>Smoking</b>', title_x=0.5)

fig.show()

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25)

print(f"Shape of X_train is: {X_train.shape}")
print(f"Shape of Y_train is: {Y_train.shape}\n")
print(f"Shape of X_test is: {X_test.shape}")
print(f"Shape of Y_test is: {Y_test.shape}")

Shape of X_train is: (41769, 8)
Shape of Y_train is: (41769, 2)

Shape of X_test is: (13923, 8)
Shape of Y_test is: (13923, 2)


In [17]:
input_dim = X.shape[1]

In [18]:
model = Sequential()
model.add(Dense(8, input_dim = input_dim, kernel_initializer='normal', activation='relu'))
model.add(Dense(5, activation = "relu", kernel_initializer='normal'))
model.add(Dense(2, activation = 'sigmoid'))

In [19]:
model.compile(optimizer = Adam(learning_rate = 0.001), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 2)                 12        
                                                                 
Total params: 129 (516.00 Byte)
Trainable params: 129 (516.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
result = model.fit(X_train, Y_train, epochs = 15, batch_size = 10)

Epoch 1/15
4177/4177 [==============================] - 13s 3ms/step - loss: 0.5710 - accuracy: 0.6867
Epoch 2/15
4177/4177 [==============================] - 10s 2ms/step - loss: 0.5019 - accuracy: 0.7136
Epoch 3/15
4177/4177 [==============================] - 10s 2ms/step - loss: 0.4953 - accuracy: 0.7180
Epoch 4/15
4177/4177 [==============================] - 9s 2ms/step - loss: 0.4931 - accuracy: 0.7204
Epoch 5/15
4177/4177 [==============================] - 10s 2ms/step - loss: 0.4934 - accuracy: 0.7211
Epoch 6/15
4177/4177 [==============================] - 9s 2ms/step - loss: 0.4917 - accuracy: 0.7226
Epoch 7/15
4177/4177 [==============================] - 9s 2ms/step - loss: 0.4907 - accuracy: 0.7225
Epoch 8/15
4177/4177 [==============================] - 9s 2ms/step - loss: 0.4915 - accuracy: 0.7235
Epoch 9/15
4177/4177 [==============================] - 9s 2ms/step - loss: 0.4908 - accuracy: 0.7224
Epoch 10/15
4177/4177 [==============================] - 10s 2ms/step - loss: 

In [22]:
y_pred =model.predict(X_test)
y_pred = (y_pred>0.5)

436/436 [==============================] - 1s 2ms/step


In [23]:
accuracy = accuracy_score(Y_test,y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.7332471450118508


In [24]:
model.save('./smoking.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



In [25]:
model = tf.keras.models.load_model('./smoking.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
open("./smoking.tflite", "wb").write(tflite_model)

2580